# **Maestría en Inteligencia Artificial Aplicada**

## **Curso: Inteligencia Artificial y Aprendizaje Automático**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## **Actividad de las Semanas 5 y 6**
### **Problema de asignación de créditos: South German Dataset.**


## **Nombres y matrículas:**



🔹 Jossie Paola Jiménez Rivera   ➡️ A01795961

🔹 Javier Augusto Rebull Saucedo ➡️ A01795838

🔹 Nancy Teresa Zapién García    ➡️ A01795907




# **Parte I: Partición, análisis y pre-procesamiento de los datos.**

## **Ejercicio 1**

In [31]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:

import numpy as np
import pandas as pd
from IPython.display import display, HTML

from sklearn.model_selection import train_test_split, cross_validate, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import make_scorer

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from xgboost import XGBClassifier

from imblearn.over_sampling import SMOTE
from imblearn.metrics import geometric_mean_score

In [33]:
# Si se desean comentar algunos de los Warnings.

import warnings
from sklearn.exceptions import ConvergenceWarning

# Suprimir los UserWarning de XGBoost relacionados con 'use_label_encoder'
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')

# Suprimir los ConvergenceWarning del MLPClassifier
warnings.filterwarnings('ignore', category=ConvergenceWarning)


In [16]:
# ************* Incluye aquí tu código: *****************************

# Cargar los datos desde el archivo especificado
df = pd.read_csv('/content/sample_data/SouthGermanCredit.asc', delimiter=' ')

# Mostrar los nombres originales de las columnas
original_columns = df.columns.tolist()
print("Nombres originales de las columnas:")
print(original_columns)

# Mostrar mensaje de éxito en color verde
display(HTML('<span style="color:green"><strong>Archivo cargado con éxito</strong></span>'))

# Lista de nombres de columnas según los nombres de variables en la Tabla 3
variable_names = [
    'Status',                  # laufkont
    'Duration',                # laufzeit
    'Credit history',          # moral
    'Purpose',                 # verw
    'Amount',                  # hoehe
    'Savings',                 # sparkont
    'Employment duration',     # beszeit
    'Installment rate',        # rate
    'Personal status sex',     # famges
    'Other debtors',           # buerge
    'Present residence',       # wohnzeit
    'Property',                # verm
    'Age',                     # alter
    'Other installment plans', # weitkred
    'Housing',                 # wohn
    'Number credits',          # bishkred
    'Job',                     # beruf
    'People liable',           # pers
    'Telephone',               # telef
    'Foreign worker',          # gastarb
    'Credit risk'              # kredit
]

# Renombrar las columnas del DataFrame
df.columns = variable_names

# Mostrar mensaje de éxito en color verde
display(HTML('<span style="color:green"><strong>Columnas renombradas con éxito</strong></span>'))

# Mostrar la correspondencia entre los nombres originales y los nuevos
print("\nCorrespondencia entre nombres originales y nuevos:\n")
for original, new in zip(original_columns, variable_names):
    print(f"{original} ---> {new}")
print("\n")

# *********** Aquí termina la sección de agregar código *************

# Mostrar las primeras filas del DataFrame transpuesto con los nuevos nombres de columnas
df.head().T

Nombres originales de las columnas:
['laufkont', 'laufzeit', 'moral', 'verw', 'hoehe', 'sparkont', 'beszeit', 'rate', 'famges', 'buerge', 'wohnzeit', 'verm', 'alter', 'weitkred', 'wohn', 'bishkred', 'beruf', 'pers', 'telef', 'gastarb', 'kredit']



Correspondencia entre nombres originales y nuevos:

laufkont ---> Status
laufzeit ---> Duration
moral ---> Credit history
verw ---> Purpose
hoehe ---> Amount
sparkont ---> Savings
beszeit ---> Employment duration
rate ---> Installment rate
famges ---> Personal status sex
buerge ---> Other debtors
wohnzeit ---> Present residence
verm ---> Property
alter ---> Age
weitkred ---> Other installment plans
wohn ---> Housing
bishkred ---> Number credits
beruf ---> Job
pers ---> People liable
telef ---> Telephone
gastarb ---> Foreign worker
kredit ---> Credit risk




,0,1,2,3,4
Status,1,1,2,1,1
Duration,18,9,12,12,12
Credit history,4,4,2,4,4
Purpose,2,0,9,0,0
Amount,1049,2799,841,2122,2171
Savings,1,1,2,1,1
Employment duration,2,3,4,3,3
Installment rate,4,2,2,3,4
Personal status sex,2,3,2,3,3
Other debtors,1,1,1,1,1


## **Ejercicio 2**

In [19]:
# Realiza a continuación una transformación para que la clase negativa (buen cliente)
# quede con el valor de 0 y la clase positiva (mal cliente) quede con el valor de 1.

# ************* Inlcuye aquí tu código:*****************************

# Transformar la columna 'credit_risk' para que los buenos clientes (1) sean 0 y los malos clientes (0) sean 1
df['Credit risk'] = df['Credit risk'].map({1: 0, 0: 1})

# *********** Aquí termina la sección de agregar código *************

print(df['Credit risk'].value_counts())

Credit risk
0    700
1    300
Name: count, dtype: int64


## **Ejercicio 3**

In [20]:
# Realiza una partición solicitada de entrenamiento y prueba.
# Los nombres de los conjuntos deberán ser como se indican en los print de abajo:

# ************* Inlcuye aquí tu código:*****************************

# Separar las características y la variable objetivo
X = df.drop('Credit risk', axis=1)
y = df['Credit risk']

# Realizar la partición en entrenamiento y prueba con los mismos porcentajes que se utilizaron en el artículo
# Supongamos que el artículo utilizó un 70% para entrenamiento y 30% para prueba

Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# *********** Aquí termina la sección de agregar código *************


# Mostremos las dimensiones de la partición generada:
print("Dimensiones:")
print("Entrenamiento:", Xtrain.shape, ytrain.shape)
print("Prueba:", Xtest.shape, ytest.shape)

# Y el porcentaje de cada clase de la variable de salida:
tmp = ytrain.sum()/ytrain.shape[0]
print("\nPorcentaje clases Positiva:%.2f%%, y Negativa:%.2f%%" % (100*(1-tmp),tmp*100))


Dimensiones:
Entrenamiento: (700, 20) (700,)
Prueba: (300, 20) (300,)

Porcentaje clases Positiva:70.00%, y Negativa:30.00%


### **Con base al porcentaje de los niveles de la variable de salida ¿podemos decir que tenemos un problema de datos desbalanceado? ¿Por qué?**



### ++++++++ Inicia la sección de agregar texto: +++++++++++

### Desbalance de Datos en el Conjunto de Entrenamiento

Sí, podemos decir que tenemos un problema de datos desbalanceados. La clase negativa (buenos clientes) representa el 70% de los datos en el conjunto de entrenamiento, mientras que la clase positiva (malos clientes) representa el 30%. Esta diferencia significativa en la proporción de clases indica un desbalance, lo cual puede afectar el rendimiento de los modelos de aprendizaje automático, especialmente aquellos que asumen que las clases están balanceadas.

Es importante abordar este desbalance utilizando técnicas de submuestreo, sobremuestreo o algoritmos que sean robustos ante datos desbalanceados. De esta manera, podemos mejorar la capacidad del modelo para identificar correctamente a los clientes de la clase minoritaria sin comprometer la precisión general.


### ++++++++ Termina la sección de agregar texto: +++++++++++



## **Ejercicio 4**

In [22]:
# De acuerdo a la información de la Tabla 3 del artículo de la IEEE
# define las variables correspondientes que se indican a continuación:

# ************* Inlcuye aquí tu código:*****************************

# Variables numéricas:
lista_paper_num = [
    'Duration',         # Duración del crédito en meses
    'Amount',           # Monto total del crédito
    'Age',              # Edad del deudor
    'People liable'     # Número total de personas dependientes
]

# Variables ordinales:
lista_paper_ord = [
    'Employment duration',     # Duración del empleo actual
    'Installment rate',        # Cuotas en función del ingreso disponible
    'Present residence',       # Duración de la residencia actual
    'Property',                # Clasificación de la propiedad
    'Number credits',          # Número de créditos existentes
    'Job'                      # Nivel de trabajo del deudor
]

# Variables nominales:
lista_paper_cat = [
    'Status',                  # Estado de la cuenta corriente
    'Credit history',          # Historial de crédito
    'Purpose',                 # Razón del crédito
    'Savings',                 # Ahorros totales del deudor
    'Personal status sex',     # Estado civil y sexo
    'Other debtors',           # Otros deudores o garantes
    'Other installment plans', # Otros planes de cuotas
    'Housing',                 # Situación de vivienda
    'Telephone',               # Disponibilidad de línea telefónica
    'Foreign worker'           # Si es trabajador extranjero
]

# Mostrar mensaje de éxito en color verde
display(HTML('<span style="color:green"><strong>Listas creadas OK</strong></span>'))

# *********** Aquí termina la sección de agregar código *************


## **Ejercicio 5**

In [28]:
# ************* Incluye aquí tu código:*****************************

# Variables numéricas:
display(HTML('<span style="color:green"><strong>Creando pipeline para variables numéricas...</strong></span>'))

numericas_pipe = Pipeline(steps=[
    ('scaler', StandardScaler())  # Normalización como se comenta en el artículo
])
numericas_pipe_nombres = lista_paper_num  # Lista de variables numéricas
print(f"Pipeline para variables numéricas creado con variables: {numericas_pipe_nombres}")
print("\n")

# Variables categóricas nominales:
display(HTML('<span style="color:green"><strong>Creando pipeline para variables categóricas nominales...</strong></span>'))
print("\n")
nominales_pipe = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))  # One-Hot-Encoding con k-1 niveles
])
nominales_pipe_nombres = lista_paper_cat  # Lista de variables nominales
print(f"Pipeline para variables categóricas nominales creado con variables: {nominales_pipe_nombres}")
print("\n")

# Variables categóricas ordinales:
display(HTML('<span style="color:green"><strong>Creando pipeline para variables categóricas ordinales...</strong></span>'))
print("\n")
ordinales_pipe = Pipeline(steps=[
    ('ordinal', OrdinalEncoder())  # Transformación OrdinalEncoder
])
ordinales_pipe_nombres = lista_paper_ord  # Lista de variables ordinales
print(f"Pipeline para variables categóricas ordinales creado con variables: {ordinales_pipe_nombres}")
print("\n")

# Conjuntar las transformaciones de todo tipo de variable:
display(HTML('<span style="color:green"><strong>Creando ColumnTransformer para aplicar las transformaciones a las variables correspondientes...</strong></span>'))
print("\n")
columnasTransformer = ColumnTransformer(transformers=[
    ('num', numericas_pipe, numericas_pipe_nombres),
    ('nom', nominales_pipe, nominales_pipe_nombres),
    ('ord', ordinales_pipe, ordinales_pipe_nombres)
])
display(HTML('<span style="color:blue"><strong>ColumnTransformer creado y listo para transformar los datos.</strong></span>'))

# *********** Aquí termina la sección de agregar código *************

Pipeline para variables numéricas creado con variables: ['Duration', 'Amount', 'Age', 'People liable']






Pipeline para variables categóricas nominales creado con variables: ['Status', 'Credit history', 'Purpose', 'Savings', 'Personal status sex', 'Other debtors', 'Other installment plans', 'Housing', 'Telephone', 'Foreign worker']






Pipeline para variables categóricas ordinales creado con variables: ['Employment duration', 'Installment rate', 'Present residence', 'Property', 'Number credits', 'Job']




## **Ejercicio 6**

In [30]:
# Como se va a utilizar Validación-Cruzada, concatena los conjuntos de entrenamiento y prueba
# en uno nuevo conjunto aumentado que llamaremos trainval para utilizar como entrenamiento:


# ************* Inlcuye aquí tu código:**************************

# Concatenar Xtrain y Xtest en Xtraintest
display(HTML('<span style="color:green"><strong>Concatenando Xtrain y Xtest para formar Xtraintest...</strong></span>'))
print("\n")
Xtraintest = pd.concat([Xtrain, Xtest], ignore_index=True)

# Concatenar ytrain y ytest en ytraintest
display(HTML('<span style="color:green"><strong>Concatenando ytrain y ytest para formar ytraintest...</strong></span>'))
print("\n")
ytraintest = pd.concat([ytrain, ytest], ignore_index=True)

# *********** Aquí termina la sección de agregar código *************


# Veamos cuántas variables nuevas se introducen con las transformaciones One-Hot-Encoding:
Xtmp = Xtraintest.copy()
tmp = columnasTransformer.fit_transform(Xtmp)
print("Dimensión de las variables de entrada ANTES de las transformaciones:", Xtmp.shape)
print("Dimensión de las variables de entrada DESPUÉS de las transformaciones:", tmp.shape)



Dimensión de las variables de entrada ANTES de las transformaciones: (1000, 20)
Dimensión de las variables de entrada DESPUÉS de las transformaciones: (1000, 41)


## **Ejercicio 7**

In [34]:
# Definimos a continuación la función que llamamos "mis_modelos" que incluye
# todos los modelos que deseamos comparar en el ejercicio.

def mis_modelos():
  modelos, nombres = list(), list()

  # ************* Incluye aquí tu código:**************************
  #
  # Deberás incluir en cada modelo los argumentos que consideres
  # adecuados para que cada uno converja y no esté sobreentrenado
  # con respecto a la métrica de la exactitud (accuracy).

  # Regresión Logística - Logistic Regression-LR:
  # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

  modelos.append(LogisticRegression(
      solver='liblinear',
      class_weight='balanced',
      max_iter=1000,
      random_state=42
  ))
  nombres.append('LR')

  # k-Vecinos más Cercanos : k-Nearest-Neighbors-kNN:
  # https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

  modelos.append(KNeighborsClassifier(
      n_neighbors=5,
      weights='uniform',
      algorithm='auto'
  ))
  nombres.append('kNN')

  # Árbol de decisiones-DecisionTree-DT:
  # https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

  modelos.append(DecisionTreeClassifier(
      max_depth=5,
      class_weight='balanced',
      random_state=42
  ))
  nombres.append('DTree')

  # Bosque Aleatorio-RandomForest-RF:
  # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

  modelos.append(RandomForestClassifier(
      n_estimators=100,
      max_depth=5,
      class_weight='balanced',
      random_state=42
  ))
  nombres.append('RF')


  # XGBoosting:
  # https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier
  # https://xgboost.readthedocs.io/en/stable/parameter.html

  modelos.append(XGBClassifier(
      use_label_encoder=False,
      eval_metric='logloss',
      n_estimators=100,
      max_depth=5,
      learning_rate=0.1,
      random_state=42
  ))
  nombres.append('XGBoost')


  # Red neuronal de Perceptrón Multicapa-MLP:
  # https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

  modelos.append(MLPClassifier(
      hidden_layer_sizes=(50,),
      activation='relu',
      solver='adam',
      max_iter=500,
      random_state=42
  ))
  nombres.append('MLP')


  # Máquina de Vectores de Soporte-SVM:
  # https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

  modelos.append(SVC(
      kernel='rbf',
      class_weight='balanced',
      probability=True,
      random_state=42
  ))
  nombres.append('SVM')

  return modelos, nombres


# Técnica de submuestreo (undersampling) y/o sobremuestreo (versampling) utilizada:
mi_uoSampling = SMOTE(random_state=42)

Xtv_uo, ytv_uo = mi_uoSampling.fit_resample(Xtraintest, ytraintest)

# *********** Aquí termina la sección de agregar código *******************

# Entrenamos cada uno de los modelos y desplegamos la métricas de Train y Val.

# NOTA: Observa que el método de Validación-Cruzada llama  a los resultados
#       de "validation" como "test":

modelos, nombres = mis_modelos()
resultados = list()

for i in range(len(modelos)):

  # Definimos nuestro pipeline con las transformaciones y los modelos:
  pipeline = Pipeline(steps=[('ct',columnasTransformer),('m',modelos[i])])

  # Aplicaremos validación-cruzada:
  micv = RepeatedStratifiedKFold(n_splits=5,
                                 n_repeats=3,
                                 random_state=5     # agreguemos una semilla para estabilizar resultados.
                                 )


  # Definimos las métricas que desamos recuperar:
  mismetricas = {'miaccuracy':'accuracy','miprecision':'precision','mirecall':'recall',
                 'mifi':'f1','miauc':'roc_auc','migmean':make_scorer(geometric_mean_score)}

  # Llevamos a cabo el entrenamiento:
  scores = cross_validate(pipeline,
                          Xtv_uo,
                          ytv_uo,
                          scoring=mismetricas,
                          cv=micv,
                          return_train_score=True,
                          )

  # Guardemos el resultado de cada modelo para análisis posteriores.
  resultados.append(scores)

  # Desplegamos los valores de las métricas para verificar si no hay
  # subentrenamiento o sobreentrenamiento:
  print('>> %s' % nombres[i])
  for j,k in enumerate(list(scores.keys())):
    if j>1:
      print('\t %s %.3f (%.3f)' % (k, np.mean(scores[k]),np.std(scores[k])))

>> LR
	 test_miaccuracy 0.797 (0.019)
	 train_miaccuracy 0.817 (0.006)
	 test_miprecision 0.788 (0.018)
	 train_miprecision 0.810 (0.005)
	 test_mirecall 0.813 (0.041)
	 train_mirecall 0.829 (0.009)
	 test_mifi 0.800 (0.022)
	 train_mifi 0.819 (0.007)
	 test_miauc 0.874 (0.017)
	 train_miauc 0.894 (0.004)
	 test_migmean 0.796 (0.019)
	 train_migmean 0.817 (0.006)
>> kNN
	 test_miaccuracy 0.774 (0.019)
	 train_miaccuracy 0.855 (0.008)
	 test_miprecision 0.746 (0.018)
	 train_miprecision 0.828 (0.009)
	 test_mirecall 0.831 (0.027)
	 train_mirecall 0.895 (0.009)
	 test_mifi 0.786 (0.019)
	 train_mifi 0.860 (0.007)
	 test_miauc 0.845 (0.016)
	 train_miauc 0.934 (0.004)
	 test_migmean 0.772 (0.019)
	 train_migmean 0.854 (0.008)
>> DTree
	 test_miaccuracy 0.766 (0.021)
	 train_miaccuracy 0.814 (0.005)
	 test_miprecision 0.725 (0.018)
	 train_miprecision 0.763 (0.008)
	 test_mirecall 0.860 (0.035)
	 train_mirecall 0.912 (0.014)
	 test_mifi 0.786 (0.021)
	 train_mifi 0.831 (0.005)
	 test_miauc

## **Ejercicio 8**

### **Escribe tus conclusiones finales de la actividad.**

### ++++++++ Inicia la sección de agregar texto: +++++++++++

### **Conclusiones Finales de la Actividad** :

En esta actividad, se evaluaron diversos modelos de aprendizaje automático para predecir el riesgo crediticio utilizando el conjunto de datos “South German Credit”. Los modelos implementados fueron:

- Regresión Logística (LR)
- k-Vecinos más Cercanos (kNN)
- Árbol de Decisión (DTree)
- Bosque Aleatorio (RF)
- XGBoost
- Perceptrón Multicapa (MLP)
- Máquina de Vectores de Soporte (SVM)

Se aplicó la técnica de sobremuestreo SMOTE para abordar el problema de desbalance de clases, donde **la clase negativa (buenos clientes) representaba el 70% y la clase positiva (malos clientes) el 30%.**

### **Análisis de Resultados**

A continuación, se presentan las métricas clave obtenidas para cada modelo en los conjuntos de prueba y entrenamiento:

1. **Regresión Logística (LR)**:
   - G-mean Test: 0.796 (0.019)
   - G-mean Train: 0.817 (0.006)
   - El modelo muestra un buen equilibrio entre las métricas de entrenamiento y prueba, lo que indica una buena generalización y ausencia de sobreajuste significativo.

2. **k-Vecinos más Cercanos (kNN)**:
   - G-mean Test: 0.772 (0.019)
   - G-mean Train: 0.854 (0.008)
   - Existe una diferencia notable entre las métricas de entrenamiento y prueba, sugiriendo un posible sobreajuste y menor capacidad de generalización.

3. **Árbol de Decisión (DTree)**:
   - G-mean Test: 0.760 (0.020)
   - G-mean Train: 0.808 (0.006)
   - El modelo podría estar subajustado debido a las bajas métricas tanto en entrenamiento como en prueba.

4. **Bosque Aleatorio (RF)**:
   - G-mean Test: 0.785 (0.020)
   - G-mean Train: 0.831 (0.007)
   - Las métricas son ligeramente superiores a las del Árbol de Decisión, indicando que el ensamble mejora el desempeño, pero aún hay espacio para optimización.

5. **XGBoost**:
   - G-mean Test: 0.823 (0.022)
   - G-mean Train: 0.966 (0.005)
   - Aunque presenta el mejor desempeño en prueba, la gran diferencia con el entrenamiento sugiere un sobreajuste significativo.

6. **Perceptrón Multicapa (MLP)**:
   - G-mean Test: 0.778 (0.023)
   - G-mean Train: 0.985 (0.003)
   - El modelo está sobreajustado, reflejado en las altas métricas de entrenamiento y menor desempeño en prueba.

7. **Máquina de Vectores de Soporte (SVM)**:
   - G-mean Test: 0.803 (0.021)
   - G-mean Train: 0.855 (0.005)
   - Muestra un buen balance entre las métricas de entrenamiento y prueba, indicando buena capacidad de generalización.

### **Observaciones Importantes**

- **Mejor Modelo**: El XGBoost alcanzó el mejor desempeño en el conjunto de prueba según la métrica G-mean. Sin embargo, el alto grado de sobreajuste es preocupante. Por otro lado, SVM y Regresión Logística ofrecen un buen equilibrio entre desempeño y generalización.
- **Sobreajuste**: Modelos como XGBoost y MLP muestran diferencias significativas entre las métricas de entrenamiento y prueba, indicando sobreajuste. Es necesario ajustar los hiperparámetros o considerar técnicas de regularización para estos modelos.
- **Desbalance de Clases**: La aplicación de SMOTE ayudó a mitigar el desbalance de clases, mejorando el desempeño de los modelos en la detección de la clase minoritaria.
- **Importancia de la Validación Cruzada**: El uso de validación cruzada estratificada con múltiples repeticiones proporcionó una evaluación más robusta del desempeño de los modelos, reduciendo la varianza asociada a una sola partición de datos.

### **Conclusión General**

La actividad permitió comparar múltiples modelos y técnicas para abordar un problema de clasificación con datos desbalanceados en el contexto del riesgo crediticio. Aunque XGBoost mostró el mejor desempeño en términos de métricas en el conjunto de prueba, su tendencia al sobreajuste sugiere que no es la opción más confiable sin ajustes adicionales.

La Máquina de Vectores de Soporte (SVM) y la Regresión Logística emergen como los modelos más balanceados, ofreciendo un buen desempeño y capacidad de generalización. Esto resalta la importancia de considerar no solo las métricas de desempeño sino también la estabilidad y robustez del modelo al momento de seleccionar la mejor opción para implementación.

Finalmente, la experiencia destaca la relevancia de manejar adecuadamente el desbalance de clases y de utilizar técnicas de validación apropiadas para obtener modelos confiables y aplicables en contextos reales.



### ++++++++ Termina la sección de agregar texto. +++++++++++


### **Referencias**

1. **Alam, T. M., Shaukat, K., Hameed, I. A., Luo, S., Sarwar, M. U., Shabbir, S., Li, J., & Khushi, M.** (2020). An investigation of credit card default prediction in the imbalanced datasets. *IEEE Access*, 8, 201173–201198. Disponible en: [https://ieeexplore.ieee.org/document/9239944](https://ieeexplore.ieee.org/document/9239944)

2. **Anzai, Y.** (2012). *Pattern Recognition and Machine Learning*. Morgan Kaufmann. Disponible en: [https://biblioteca.tec.mx/oreilly](https://biblioteca.tec.mx/oreilly)

3. **Hu, Y., Yang, L., Chen, L., & Zhu, S.** (2020). Research on a customer churn combination prediction model based on decision tree and neural network. En *2020 IEEE 5th International Conference on Cloud Computing and Big Data Analytics (ICCCBDA)* (pp. 129–132). IEEE. [https://doi.org/10.1109/ICCCBDA49378.2020.9095611](https://doi.org/10.1109/ICCCBDA49378.2020.9095611)

4. **Tecnológico de Monterrey.** (2024). *MNA_IAyAA_semana_5_y_6_Actividad_sept_2024.pdf*. [Material de clase]. Curso Inteligencia Artificial y Aprendizaje Automático (Grupo 10), impartido por el Dr. Luis Eduardo Falcón Morales.

5. **Tecnológico de Monterrey.** (2024). *MNA_IAyAA_Titanic_semanas_5y6.ipynb*. [Material complementario]. Curso Inteligencia Artificial y Aprendizaje Automático (Grupo 10), impartido por el Dr. Luis Eduardo Falcón Morales.

6. **Tecnológico de Monterrey.** (2024). *MNA_IAyAA_semana_6_Arboles_de_decision_y_BosqueAleatorio_teoria-1.pdf*. [Material de clase]. Curso Inteligencia Artificial y Aprendizaje Automático (Grupo 10), impartido por el Dr. Luis Eduardo Falcón Morales.

7. **Universidad de California, Irvine.** (s.f.). *South German Credit Data Set*. Recuperado en octubre de 2024 de [https://archive.ics.uci.edu/dataset/522/south+german+credit](https://archive.ics.uci.edu/dataset/522/south+german+credit)





>> ### **Fin de la Actividad de las Semanas 5 y 6.**